In [1]:
from ETGEMs_function_PMI import *
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy

In [2]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './reaction_g0_strain0.txt'
reaction_g0_file_original1 = './reaction_g0_strain1.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './metabolites_lnC_strain0.txt'
metabolites_lnC_file_original1 = './metabolites_lnC_strain1.txt'
#Get Model from local file
model_file_original0 = './iML1515.xml'
model_file_original1 = './iMM904.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_strain0.csv'
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_strain1.csv'

In [3]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)
## knockout the fhuA gene
model0['lb_list']['EX_fecrm_e ']=0
model0['lb_list']['EX_fecrm_e ']=0

In [4]:
# get the information of km, vmax and public metabolites
km = pd.read_csv('./polymicroorganism/km.csv')
vmax = pd.read_csv('./polymicroorganism/vmax.csv')
public_metabolism = pd.read_csv('./polymicroorganism/public_metabolism_test.csv')

'''
public_metabolism_name_list = []
public_metabolism_concentration_list = []
for i in public_metabolism['metabolism']:
    public_metabolism_name_list.append(i)
for j in public_metabolism['concentration']:
    public_metabolism_concentration_list.append(j)
public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
print(public_metabolism)
'''

"\npublic_metabolism_name_list = []\npublic_metabolism_concentration_list = []\nfor i in public_metabolism['metabolism']:\n    public_metabolism_name_list.append(i)\nfor j in public_metabolism['concentration']:\n    public_metabolism_concentration_list.append(j)\npublic_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))\nprint(public_metabolism)\n"

In [5]:
# definate the function to simulate metabolism and interaction
def time_space_state(model_list, biomass_list, growth_list, breed_list, parameter_list, public_metabolism, vmax, km, deta_t, deta_s, micro_distribute_threshold, length, D):
    
    number_cell_side = length // deta_s
    number_cell_side = int(number_cell_side)
    public_metabolism_name_list = []
    public_metabolism_concentration_list = []
    for i in public_metabolism['metabolism']:
        public_metabolism_name_list.append(i)
    for j in public_metabolism['concentration']:
        public_metabolism_concentration_list.append(j)
    public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
    
    number_model = len(model_list)
    
    k_m = {}
    v_max = {}
    for i in range(number_model):
        for j in range(len(km['reactions_strain'+str(i)])):
            k_m[(i, km['reactions_strain'+str(i)][j])] = km['km_strain'+str(i)][j]
            v_max[(i, vmax['reactions_strain'+str(i)][j])] = vmax['vmax_strain'+str(i)][j]
    
    
    number_public_metabolism = len(public_metabolism)
    distribute_micro_list = {}
    distribute_public_metabolism_list = {}
    distribute_lb_list = {}
    public_metabolism_r_list = []
    #set the initial distribution of straints
    for i in range(number_model):
        distribute_micro = np.random.randint(20, size=number_cell_side)
        distribute_micro_list.update({i: distribute_micro})
    print(distribute_micro_list)
    #set the initial distribution of metabolites
    for i in public_metabolism_name_list:
        distribute_public_metabolism = multiply(np.mat(np.ones(number_cell_side)), public_metabolism[i])
        distribute_public_metabolism_list.update({i: distribute_public_metabolism})
    #calculate the upperbounds of uptake reactions for public metabolites
    public_reaction_ub_list = {}
    public_reaction_list = {}
    for i in range(number_model):
        public_reaction_ub = {}
        public_reaction = []
        for rea in model_list[i]['reaction_list']:
            if 'EX_' in rea:
                for j in public_metabolism_name_list:
                    try:
                        model_list[i]['coef_matrix'][(j,rea)]
                    except:
                        pass
                    else:
                        ub = np.mat(np.ones(number_cell_side))
                        if model_list[i]['coef_matrix'][(j,rea)] > 0:
                            for m in range(number_cell_side):
                                ub[0,m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                        else:
                            try:
                                model_list[i]['ub_list'][rea]
                            except:
                                ub = ub * 1000
                            else:
                                ub = ub * model_list[i]['ub_list'][rea]
                        public_reaction_ub.update({rea: ub})
                        public_reaction.append(rea)
                        break
            public_reaction_ub_list[i] = public_reaction_ub
            public_reaction_list[i] = public_reaction

            
    ct = 0
    
    distribute_micro_list_t = {ct: distribute_micro_list}
    distribute_public_metabolism_list_t = {ct: distribute_public_metabolism_list}
    distribute_lb_list_t = {ct: distribute_lb_list}
    r = micro_distribute_threshold + 1
    r_threshold = [r]*5
    slip_r = np.mean(r_threshold[-5:])
    number = {}
    various = {}
    for i in range(number_model):
        number[i] = [np.mean(distribute_micro_list[i])]
        various[i] = [np.std(distribute_micro_list[i])/np.mean(distribute_micro_list[i])]
    
            
    # iterative simulation by slip_r
    while slip_r >= micro_distribute_threshold:
        ct += 1
        print(ct)
        distribute_micro_list_t[ct] = copy.deepcopy(distribute_micro_list_t[ct-1])
        micro_decrease = {}
        micro_increase = {}
        
        #simulate the cell wandering
        #micro_decrease: the decrease number of strains
        #micro_increase: the increase number of strains
        #micro_decrease_cell: the decrease number of strains in the current grid
        #micro_increase_fcell: the increase number of strains in the forward grid
        #micro_increase_bcell: the increase number of strains in the backward grid
        for m in range(number_cell_side):
            micro_decrease_cell = {}
            micro_increase_cell = {}
            for i in range(number_model):
                micro_decrease_cell[i] = 0
                micro_increase_cell[i] = 0
            micro_decrease[m] = micro_decrease_cell
            micro_increase[m] = micro_increase_cell
            
            
        if ct > 0:
            met = 'glc__D_e'
            threshold = 0.3
            for m in range(number_cell_side):
                #calculate the number of strains in the internal grids
                if 0<m<number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = int(micro_decrease_cell)
                                    micro_increase_fcell_ratio = (distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * micro_decrease_cell
                                    micro_increase_fcell = int(micro_increase_fcell)
                                    micro_increase_bcell = micro_decrease_cell-micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                                    micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                                else:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = int(micro_decrease_cell)
                                    micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_increase_fcell = int(micro_increase_fcell)
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                            elif distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = int(micro_decrease_cell)
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = int(micro_increase_bcell)
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the first grid
                elif m == 0:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = int(micro_decrease_cell)
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = int(micro_increase_bcell)
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the last grid
                elif m == number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = int(micro_decrease_cell)
                                micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_fcell = int(micro_increase_fcell)
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
        
            #calculate the number of strains after wandering
            for m in range(number_cell_side):
                for i in range(number_model):
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] - micro_decrease[m][i]
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] + micro_increase[m][i]
                    distribute_micro_list_t[ct][i][m] = max(0, distribute_micro_list_t[ct][i][m])
                            
                            
        
        #simulate the substrate diffusion by Fick's second law
        for m in range(number_cell_side): 
            if 0<m<number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + ((distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_s-(distribute_public_metabolism_list[met][0,m]-distribute_public_metabolism_list[met][0,m-1])/deta_s)/deta_s*D*deta_t
                    if distribute_public_metabolism_list[met][0,m] < 0:
                        print('Warning: the D is too high!')
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == 0:
                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_s/deta_s*D*deta_t
                distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == number_cell_side-1:
                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m-1]-distribute_public_metabolism_list[met][0,m])/deta_s/deta_s*D*deta_t
                distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
        
        #simulate the metabolism by ETMs
        biomass_value_list = {}
        number_model_range = []
        for m in range(number_cell_side):
            if m%2 == 0:
                number_model_range.append([0,1])
            elif m%2 == 1:
                number_model_range.append([1,0])
        for m in range(number_cell_side):
            B_value_list = []
            
            
            biomass_value_micro = {}
            for i in number_model_range[m]:
                if distribute_micro_list_t[ct][i][m] > 0:
                    public_metabolism_flux_list = {}
                    for j in public_metabolism_name_list:
                        public_metabolism_flux_list.update({j: 0})
                    for j in public_reaction_list[i]:
                        model_list[i]['ub_list'][j] = int(public_reaction_ub_list[i][j][0,m])
            
                    biomass_id = biomass_list[i]
                    E_total=parameter_list[i][0]
                    #set the carbon source to glucose
                    substrate_name='EX_glc__D_e_reverse'
                    substrate_value=parameter_list[i][1]
                    biomass_value=growth_list[i]
                    K_value=parameter_list[i][2]

                    try:
                        MDF_Calculation(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,'gurobi')
                    except:
                        pass
                    else:
                        #calculate the MDF values of metabolic networks
                        biomass_value_micro[i] = biomass_value
                        B_value=MDF_Calculation(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,'gurobi')
                        B_value_list.append(B_value)
                        #calculate the biomass yield under the MDF value
                        obj_name=biomass_list[i]
                        obj_target='maximize'
                        if i == 0:
                            max_biomass_under_mdf=Max_Growth_Rate_Calculation0(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
                        elif i == 1:
                            max_biomass_under_mdf=Max_Growth_Rate_Calculation1(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
                        biomass_value=max_biomass_under_mdf*0.9
                        
                        #calculate the minimum value of the sum of the fluxes
                        if i == 0:
                            [min_V,Concretemodel]=Min_Flux_Sum_Calculation0(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
                        elif i == 1:
                            [min_V,Concretemodel]=Min_Flux_Sum_Calculation1(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
            
                        #translating the results of ETMs into a usable form
                        model=model_list[i]['model']
                        reaction_kcat_MW=model_list[i]['reaction_kcat_MW']
                        reaction_g0=model_list[i]['reaction_g0']
                        coef_matrix=model_list[i]['coef_matrix']
                        metabolite_list=model_list[i]['metabolite_list']
                        use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
            
                        #simulate the fluxes of the public metabolites
                        for rea in public_reaction_list[i]:
                            for met in public_metabolism_name_list:
                                try:
                                    model_list[i]['coef_matrix'][(met,rea)]  
                                except:
                                    pass
                                else:
                                    public_metabolism_flux_list[met] = public_metabolism_flux_list[met] + model_list[i]['coef_matrix'][(met,rea)]* use_result['flux'][rea]
                        
                        #simulate the distribution of the public metabolites
                        distribute_public_metabolism_ori = copy.deepcopy(distribute_public_metabolism_list)
                        distribute_public_metabolism_re = copy.deepcopy(distribute_public_metabolism_ori)
                        for met in public_metabolism_name_list:
                            distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m]
                            distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                            if distribute_public_metabolism_list[met][0,m] < 0:
                                print(met+'_1: ', distribute_public_metabolism_list[met][0,m])
                                
                        #simulate the multiplication and death rates of strains and the distribution of public metabolites after multiplication or death        
                        death_rate = 0
                        birth_rate = 1
                        for rea in public_reaction_list[i]:
                            if 'reverse' not in rea:
                                met = rea[3::]
                                if met in public_metabolism_name_list:
                            
                                    intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                    intracellular_c = intracellular_c.replace(';',',"')
                                    intracellular_c = intracellular_c.replace(' :','" :')
                                    intracellular_c = '{"' + intracellular_c + '}'
                                    intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                    bd_rate_met = (distribute_public_metabolism_list[met][0,m] - 0.1)/intracellular_c_dict[met]/distribute_micro_list_t[ct][i][m]
                                    if bd_rate_met < 0:
                                        death_rate_lb = (public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] - distribute_public_metabolism_re[met][0,m] + 0.1)/(public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] + intracellular_c_dict[met] * distribute_micro_list_t[ct][i][m])
                                        death_rate = max(death_rate, death_rate_lb)
                                    else:
                                        birth_rate = min(birth_rate, bd_rate_met)
                        death_rate = min(death_rate, 1)
                        birth_rate = min(birth_rate,1)
                        if death_rate > 0:
                            birth_rate = 0
                            for met in public_metabolism_name_list:
                                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + public_metabolism_flux_list[met] * deta_t * math.ceil(distribute_micro_list_t[ct][i][m] * death_rate)
                                distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                    
                        for rea in public_reaction_list[i]:
                            if 'reverse' not in rea:
                                met = rea[3::]
                                if met in public_metabolism_name_list:
                            
                                    intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                    intracellular_c = intracellular_c.replace(';',',"')
                                    intracellular_c = intracellular_c.replace(' :','" :')
                                    intracellular_c = '{"' + intracellular_c + '}'
                                    intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_ori[met][0,m] - intracellular_c_dict[met]*math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i]) + intracellular_c_dict[met]*math.ceil(death_rate*distribute_micro_list_t[ct][i][m])
                                    if distribute_public_metabolism_list[met][0,m] < 0:
                                        print(met+'_2: ', distribute_public_metabolism_list[met][0,m])
                                        distribute_public_metabolism_list[met][0,m] = 0
                        
                        #simulate the distribution of strains after multiplication or death
                        distribute_micro_increase = math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i])
                        distribute_micro_decrease = math.ceil(distribute_micro_list_t[ct][i][m]*death_rate)
                        distribute_micro_list_t[ct][i][m] = max(0,distribute_micro_list_t[ct][i][m] + distribute_micro_increase - distribute_micro_decrease)
            
            #calculate the upperbounds of nutrient uptake rates after substrate diffusion, cell wandering, metabolism, multiplication and death
            biomass_value_list[m] = biomass_value_micro
            
            for i in range(number_model):
                for rea in model_list[i]['reaction_list']:
                    if 'EX_' in rea:
                        for j in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(j,rea)]
                            except:
                                pass
                            else:
                                if model_list[i]['coef_matrix'][(j,rea)] > 0:
                                    public_reaction_ub_list[i][rea][0, m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
        
        # calculate the mean number and uniformity of distribution of strains at this iteration
        for i in range(number_model):
            strain_number = np.mean(distribute_micro_list_t[ct][i])
            strain_various = np.std(distribute_micro_list_t[ct][i])
            number[i].append(strain_number)
            various[i].append(strain_various)
            print('strain_number: ', i, strain_number)
            print('strain_various: ', i, strain_various)
        
        # calculate the slip_r at this iteration
        if ct > 1:
            r = 0
            for i in range(number_model):
                for m in range(number_cell_side):
                    if distribute_micro_list_t[ct-1][i][m] > 0:
                        r = r + ((distribute_micro_list_t[ct][i][m]-distribute_micro_list_t[ct-1][i][m])/(distribute_micro_list_t[ct-1][i][m]))**2
                    else:
                        r = r + (distribute_micro_list_t[ct][i][m])**2
        
        r_threshold.append(r)
        slip_r = np.mean(r_threshold[-5:])
        fd_r_threshold = r_threshold[5:]
        print('fd_r_threshold: ', fd_r_threshold)
        print('slip_r: ', slip_r)
    return(distribute_micro_list_t, number, various)

In [6]:
distribute_micro_list_t, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [4, 6], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/30, 0.1, 0.05, 2, 0.2)
print('The change process of microorganism distribution is: ',distribute_micro_list_t)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([19, 10,  7, 12, 18,  8, 13, 11,  5,  3,  1,  4,  0, 18,  7,  0,  7,
        2, 12]), 1: array([11,  8,  5,  9, 18,  9,  2,  9,  2,  2,  5, 17, 16, 17,  2,  7, 17,
        7,  0])}
1
strain_number:  0 9.947368421052632
strain_various:  0 7.26544586388943
strain_number:  1 9.526315789473685
strain_various:  1 6.659922720056075
fd_r_threshold:  [1.05]
slip_r:  1.05
2
strain_number:  0 12.105263157894736
strain_various:  0 6.560606012140753
strain_number:  1 10.68421052631579
strain_various:  1 5.140680188242013
fd_r_threshold:  [1.05, 89.50866582150499]
slip_r:  18.741733164301
3
strain_number:  0 14.789473684210526
strain_various:  0 7.466267632128561
strain_number:  1 11.947368421052632
strain_various:  1 4.160891658116288
fd_r_threshold:  [1.05, 89.50866582150499, 20.763139240083127]
slip_r:  22.684361012317623
4
strain_number:  0 18.210526315789473
strain_various:  0 8.426642881320383
strain_number:  1 13.631578947368421
strain_various:  1 3.6159082183826667
fd_r_threshold:

glc__D_e_1:  -1.2306011997997786
glc__D_e_1:  -0.999430926091351
glc__D_e_1:  -7.858042479575531
glc__D_e_1:  -5.823509148570284
glc__D_e_1:  -0.49377477645295564
glc__D_e_1:  -1.98970971660738
glc__D_e_1:  -3.1756244643203733
glc__D_e_1:  -2.5468991252605804
glc__D_e_1:  -2.936827903245987
glc__D_e_1:  -3.2067645875963593
glc__D_e_1:  -4.484131172841466
glc__D_e_1:  -0.37049402402542153
glc__D_e_1:  -1.8349717264334522
glc__D_e_1:  -4.023897546795212
glc__D_e_1:  -1.2144394061124113
glc__D_e_1:  -0.3748868874332798
glc__D_e_1:  -4.0001545035085115
glc__D_e_1:  -5.034635902069477
glc__D_e_1:  -1.011574804738296
glc__D_e_1:  -3.368060779215921
glc__D_e_1:  -4.645427364461028
glc__D_e_1:  -1.3694657811030329
glc__D_e_1:  -1.7593945590884394
glc__D_e_1:  -2.4798930096366867
glc__D_e_1:  -3.757259594881794
strain_number:  0 11.578947368421053
strain_various:  0 5.556558850618592
strain_number:  1 9.0
strain_various:  1 4.376973239219325
fd_r_threshold:  [1.05, 89.50866582150499, 20.7631392

glc__D_e_1:  -0.7849596835681794
glc__D_e_1:  -0.6163843211565467
glc__D_e_1:  -0.23701828748560494
glc__D_e_1:  -0.662807149233974
glc__D_e_1:  -0.0807220236933921
glc__D_e_1:  -1.0472063641851672
glc__D_e_1:  -0.23480695687917486
glc__D_e_1:  -1.2545966953311465
glc__D_e_1:  -0.41835754629795563
glc__D_e_1:  -1.9209482002396614
glc__D_e_1:  -1.2810957373874525
glc__D_e_1:  -0.4398445873205312
glc__D_e_1:  -0.1969594006363895
glc__D_e_1:  -0.4358186988068904
glc__D_e_1:  -1.291327232432903
glc__D_e_1:  -1.4742309074971303
strain_number:  0 4.578947368421052
strain_various:  0 1.3885164166603075
strain_number:  1 3.6842105263157894
strain_various:  1 1.4528603938781326
fd_r_threshold:  [1.05, 89.50866582150499, 20.763139240083127, 18.471655483187032, 10.21065431188961, 7.330742406089969, 5.051594358573395, 3.4742224956942085, 3.120942076695434, 3.21670273595247, 3.0491090834838315, 3.1374690824925238, 6.88496650379193, 5.613821583767055, 2.7351512702942524, 4.248224148894728, 3.5960175

glc__D_e_1:  -0.37277233592761183
glc__D_e_1:  -0.16420897743428997
glc__D_e_1:  -0.14604903525945967
glc__D_e_1:  -0.04116115576241519
strain_number:  0 2.1052631578947367
strain_various:  0 0.7177990366834661
strain_number:  1 0.631578947368421
strain_various:  1 0.7405919620773836
fd_r_threshold:  [1.05, 89.50866582150499, 20.763139240083127, 18.471655483187032, 10.21065431188961, 7.330742406089969, 5.051594358573395, 3.4742224956942085, 3.120942076695434, 3.21670273595247, 3.0491090834838315, 3.1374690824925238, 6.88496650379193, 5.613821583767055, 2.7351512702942524, 4.248224148894728, 3.596017573696145, 7.5632427878558826, 2.600539965986395, 2.412908163265306, 2.4533333333333336, 3.319722222222222, 2.4791666666666665, 2.2969444444444447, 3.430555555555556, 1.8333333333333333, 2.6944444444444446]
slip_r:  2.546888888888889
28
glc__D_e_1:  -0.17585574140301752
glc__D_e_1:  -0.11516307310473889
glc__D_e_1:  -0.0661117664389087
glc__D_e_1:  -0.16910639103420722
glc__D_e_1:  -0.039088

strain_number:  0 0.47368421052631576
strain_various:  0 0.4993069989739547
strain_number:  1 0.15789473684210525
strain_various:  1 0.36464227527765836
fd_r_threshold:  [1.05, 89.50866582150499, 20.763139240083127, 18.471655483187032, 10.21065431188961, 7.330742406089969, 5.051594358573395, 3.4742224956942085, 3.120942076695434, 3.21670273595247, 3.0491090834838315, 3.1374690824925238, 6.88496650379193, 5.613821583767055, 2.7351512702942524, 4.248224148894728, 3.596017573696145, 7.5632427878558826, 2.600539965986395, 2.412908163265306, 2.4533333333333336, 3.319722222222222, 2.4791666666666665, 2.2969444444444447, 3.430555555555556, 1.8333333333333333, 2.6944444444444446, 2.8125, 0.7222222222222222, 3.4722222222222223, 2.25, 5.5, 2.25, 1.0, 0.25, 3.0, 0.0]
slip_r:  1.3
38
strain_number:  0 0.47368421052631576
strain_various:  0 0.4993069989739547
strain_number:  1 0.15789473684210525
strain_various:  1 0.36464227527765836
fd_r_threshold:  [1.05, 89.50866582150499, 20.763139240083127, 1

strain_number:  0 0.10526315789473684
strain_various:  0 0.3068922049918579
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.05, 89.50866582150499, 20.763139240083127, 18.471655483187032, 10.21065431188961, 7.330742406089969, 5.051594358573395, 3.4742224956942085, 3.120942076695434, 3.21670273595247, 3.0491090834838315, 3.1374690824925238, 6.88496650379193, 5.613821583767055, 2.7351512702942524, 4.248224148894728, 3.596017573696145, 7.5632427878558826, 2.600539965986395, 2.412908163265306, 2.4533333333333336, 3.319722222222222, 2.4791666666666665, 2.2969444444444447, 3.430555555555556, 1.8333333333333333, 2.6944444444444446, 2.8125, 0.7222222222222222, 3.4722222222222223, 2.25, 5.5, 2.25, 1.0, 0.25, 3.0, 0.0, 0.0, 0.0, 2.0, 3.0, 1.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0]
slip_r:  0.6
49
strain_number:  0 0.10526315789473684
strain_various:  0 0.3068922049918579
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.05, 89.50866582150499, 20.763139240083127, 18.4716554